In [51]:
from sklearn import linear_model
import numpy as np

In [20]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [55]:
def regression_linear_models(train_set_ready, train_set_labels, mod=[], cv=4,
                             linreg_attr=[], ridge_attr=[], lasso_attr=[],
                             elnet_attr=[], lars_attr=[], lassolars_attr=[],
                             orthmatch_attr=[], bayes_attr=[], logreg_attr=[],
                             sgd_attr=[], passagr_attr=[], huber_attr=[]):
    
    models = {'LinearRegression': {'rmse_training': -1, 'rmse_cv': -1, 'attr':linreg_attr },
                   'Ridge': {'rmse_training': -1, 'rmse_cv': -1, 'attr':ridge_attr },
                   'Lasso': {'rmse_training': -1, 'rmse_cv': -1, 'attr':lasso_attr },
                   'ElasticNet': {'rmse_training': -1, 'rmse_cv': -1, 'attr':elnet_attr },
                   'Lars': {'rmse_training': -1, 'rmse_cv': -1, 'attr':lars_attr },
                   'LassoLars': {'rmse_training': -1, 'rmse_cv': -1, 'attr':lassolars_attr },
                   'OrthogonalMatchingPursuit': {'rmse_training': -1, 'rmse_cv': -1, 'attr':orthmatch_attr },
                   'BayesianRidge': {'rmse_training': -1, 'rmse_cv': -1, 'attr':bayes_attr },
                   'LogisticRegression': {'rmse_training': -1, 'rmse_cv': -1, 'attr':logreg_attr },
                   'SGDRegressor': {'rmse_training': -1, 'rmse_cv': -1, 'attr':sgd_attr },
                   'PassiveAggressiveRegressor': {'rmse_training': -1, 'rmse_cv': -1, 'attr':passagr_attr },
                   'HuberRegressor': {'rmse_training': -1, 'rmse_cv': -1, 'attr':huber_attr }
                  }

    train_set_labels = np.ravel(train_set_labels)
    for k,v in sorted(models.items()):
        if k in mod:
            print(k,': ')
            fun = getattr(linear_model, k)
            #reg = fun(models[k]['attr'])
            reg = fun()
            reg.fit(train_set_ready, train_set_labels)

            predictions = reg.predict(train_set_ready)

            train_mse = mean_squared_error(predictions, train_set_labels)
            v['rmse_training'] = np.sqrt(train_mse)

            scores = cross_val_score(reg, train_set_new_ready, np.ravel(train_set_labels),
                                          scoring="neg_mean_squared_error", cv=cv)

            v['rmse_cv'] = np.sqrt(-scores).mean()
        
    return models